In [1]:
import numpy as np
import matplotlib.pyplot as plt
import string
import random
import re
import requests
import os
import textwrap

In [2]:
### Create substitution cipher

# one will act as the key, other as the value

letters1 = list(string.ascii_lowercase)
letters2 = list(string.ascii_lowercase)

true_mapping = {}

# shuffle second set of letters
random.shuffle(letters2)

# Populate map
for k, v in zip(letters1, letters2):
    true_mapping[k] = v

In [3]:
true_mapping

{'a': 'k',
 'b': 'n',
 'c': 'b',
 'd': 'u',
 'e': 'l',
 'f': 'y',
 'g': 'm',
 'h': 'o',
 'i': 'z',
 'j': 'a',
 'k': 'g',
 'l': 'x',
 'm': 'c',
 'n': 'p',
 'o': 'w',
 'p': 'v',
 'q': 'd',
 'r': 'r',
 's': 'e',
 't': 's',
 'u': 'q',
 'v': 'j',
 'w': 'h',
 'x': 'i',
 'y': 't',
 'z': 'f'}

In [4]:
### The language model

# Initialize Markov matrix
M = np.ones((26, 26))

# initialize state distributaion
pi = np.zeros(26)

# a function to update the markov matrix
def update_transition(ch1, ch2):
    # ord('a') = 97, ord('b') = 98, ...
    i = ord(ch1)-97
    j = ord(ch2)-97
    M[i, j] += 1

# function to update the initial state distribution
def update_pi(ch):
    i = ord(ch)-97
    pi[i] += 1
    
# get the log-probability of a word / token
def get_word_prob(word):
    i = ord(word[0]) - 97
    logp = np.log(pi[i])
    
    for ch in word[1:]:
        j = ord(ch) - 97
        logp += np.log(M[i, j]) # update prob
        i = j # update j
        
    return logp

# get the probability of a sequence of words
def get_sequence_prob(words):
    # if input is string, split into an array of tokens
    if type(words) == str:
        words = words.split()
        
    logp = 0
    for word in words:
        logp = logp + get_word_prob(word)
    return logp

In [5]:
### Create markov model based on an English dataset
# is an edit of https://www.gutenberg.org/ebooks/2701 
# (removed the frond and back matter)

# download the file
# if not os.path.exists('moby_dick.txt'):
#     print("Downloading moby dick...")
#     r = requests.get('https://lazyprogrammer.me/course_files/moby_dick.txt')
# with open('moby_dick.txt', 'w') as f:
#     f.write(r.content.decode())

In [6]:
# for replacing non-alpha characters
regex = re.compile('[^a-zA-Z]')

# load in words
for line in open('moby_dick.txt'):
    line = line.rstrip()
    
    # There are blank line in the file.
    if line:
        line = regex.sub(' ', line) # replace all non-alpha characters with space
        
        # split the tokens in the line and lowercase
        tokens = line.lower().split()
        
        for token in tokens:
            # update the model
            # first letter
            ch0 = token[0]
            update_pi(ch0)
            
            # other letters
            for ch1 in token[1:]:
                update_transition(ch0, ch1)
                ch0 = ch1
        
    # normalize the probabilities
    pi /= pi.sum()
    M /= M.sum(axis=1, keepdims=True)

In [7]:
### Encode a message

# Random exerpt from Project Gutenberg's
# The adventures of Sherlock Holmes, by Arthur conan Dyle
# https://www.gutenberg.org/ebooks/1661

original_message = '''I then lounged down the street and found, 
as I expected, the there was a mews in a lane which runs down
by one wall of the garden. I lent the ostlers a hand in rubbing
down their horses, and received in exchange twopence, a glass of 
half-and-half, two fills of shag tobacco, and as much information
as I could desire about Miss Adler, to say nothing of half a dozen 
other people in the neighbourhood in whom I was not in the least
interested, but whose biographies I was compelled to listen to.
'''



In [8]:
# Function to encode a message
def encode_message(msg):
    # downcase
    msg = msg.lower()
    
    # replace non-alpha characters
    msg = regex.sub(' ', msg)
    
    # make the encoded message
    coded_msg = []
    for ch in msg:
        coded_ch = ch # could be just space or punctuation
        if ch in true_mapping:
            coded_ch = true_mapping[ch]
        coded_msg.append(coded_ch)
    return ''.join(coded_msg)

encoded_message = encode_message(original_message)

# A function to decode a message
def decode_message(msg, word_map):
    decoded_msg = []
    for ch in msg:
        decoded_ch = ch # could be just space or punctuation
        if ch in word_map:
            decoded_ch = word_map[ch]
        decoded_msg.append(decoded_ch)
        
    return ''.join(decoded_msg)

In [9]:
encoded_message

'z solp xwqpmlu uwhp sol esrlls kpu ywqpu   ke z livlbslu  sol solrl hke k clhe zp k xkpl hozbo rqpe uwhp nt wpl hkxx wy sol mkrulp  z xlps sol wesxlre k okpu zp rqnnzpm uwhp solzr owrele  kpu rlblzjlu zp libokpml shwvlpbl  k mxkee wy  okxy kpu okxy  shw yzxxe wy eokm swnkbbw  kpu ke cqbo zpywrckszwp ke z bwqxu ulezrl knwqs czee kuxlr  sw ekt pwsozpm wy okxy k uwflp  wsolr vlwvxl zp sol plzmonwqrowwu zp howc z hke pws zp sol xlkes zpslrleslu  nqs howel nzwmrkvozle z hke bwcvlxxlu sw xzeslp sw  '

In [10]:
decoded_message = decode_message(encoded_message, true_mapping)
decoded_message

'f ewxv ihdvcxq qhov ewx lerxxe gvq thdvq   gl f xzjxnexq  ewx ewxrx ogl g bxol fv g igvx owfnw rdvl qhov ps hvx ogii ht ewx cgrqxv  f ixve ewx hleixrl g wgvq fv rdppfvc qhov ewxfr whrlxl  gvq rxnxfaxq fv xznwgvcx eohjxvnx  g cigll ht  wgit gvq wgit  eoh tfiil ht lwgc ehpgnnh  gvq gl bdnw fvthrbgefhv gl f nhdiq qxlfrx gphde bfll gqixr  eh lgs vhewfvc ht wgit g qhyxv  hewxr jxhjix fv ewx vxfcwphdrwhhq fv owhb f ogl vhe fv ewx ixgle fvexrxlexq  pde owhlx pfhcrgjwfxl f ogl nhbjxiixq eh iflexv eh  '

In [11]:
### Run an evolutionary algorithm to decode the message

# this is our initilaization point
dna_pool = []
for _ in range(20):
    dna = list(string.ascii_lowercase)
    random.shuffle(dna)
    dna_pool.append(dna)
    

In [12]:
def evolve_offspring(dna_pool, n_children):
    # make n_children per offspring
    offspring = []
    for dna in dna_pool:
        for _ in range(n_children):
            copy = dna.copy()
            j = np.random.randint(len(copy))
            k = np.random.randint(len(copy))
            
            # switch
            tmp = copy[j]
            copy[j] = copy[k]
            copy[k] = tmp
            offspring.append(copy)
        
    return offspring + dna_pool

    

In [13]:
num_iters = 1000
scores = np.zeros(num_iters)
best_dna = None
best_map = None
best_score = float('-inf')
for i in range(num_iters):
    if i>0:
        # get offspring from the current dna_pool
        dna_pool = evolve_offspring(dna_pool, 3)
        
    # calculate the score for each dna
    dna2score = {}
    for dna in dna_pool:
        # populate map
        current_map = {}
        for k, v in zip(letters1, dna):
            current_map[k] = v
        
        decoded_message = decode_message(encoded_message, current_map)
        score = get_sequence_prob(decoded_message)
        
        # store it
        # Needs to be a string to be a dict key
        dna2score[''.join(dna)] = score
        
        # record the best so far
        if score > best_score:
            best_dna = dna
            best_map = current_map
            best_score = score
        
    # average score for this generation
    scores[i] = np.mean(list(dna2score.values()))
        
    # keep the best 5 dna
    # also turn them back into list of single chars
    sorted_dna = sorted(dna2score.items(), key=lambda x: x[1], reverse=True)
    dna_pool = [list(k) for k, v in sorted_dna[:5]]
        
    if i % 200 == 0:
        print("iter: ", i, "Score: ", scores[i], "best so far: ", best_score )

C:\Users\hp\AppData\Local\Temp/ipykernel_6604/1722455342.py:24: RuntimeWarning: divide by zero encountered in log
  logp = np.log(pi[i])


iter:  0 Score:  -inf best so far:  -inf
iter:  200 Score:  -inf best so far:  -inf
iter:  400 Score:  -inf best so far:  -inf
iter:  600 Score:  -inf best so far:  -inf
iter:  800 Score:  -inf best so far:  -inf


In [14]:
# Use best score
decoded_message = decode_message(encoded_message, best_map)
print("LL of decoded message: ", get_sequence_prob(decoded_message))
print("LL of true message: ", get_sequence_prob(regex.sub(regex.sub(' ', original_message.lower()))))

# Which letters are wrong?
for true, v in true_mapping.items():
    pred = best_map[v]
    if true != pred:
        print("true: %s, pred: %s" %(true, pred))
        

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
# Print the final decoded message
print("Decoded message: \n", textwrap.fill(decoded_message))
print("\nOriginal message:\n", original_message)